 ## Environment Setup & Imports

In [1]:
import os

os.makedirs('src', exist_ok=True)
os.makedirs('kernels', exist_ok=True)
os.makedirs('notebooks', exist_ok=True)

print("✅ Directories created: src, kernels, notebooks")

with open('src/__init__.py', 'w') as f:
    pass


✅ Directories created: src, kernels, notebooks


In [2]:
%%writefile src/utils.py
from typing import Tuple
import time
import numpy as np
import cupy as cp

def generate_matrices(n: int, dtype=np.float32) -> Tuple[np.ndarray, np.ndarray]:
    """
    Generates two random N*N matrices A and B.
    Using float32 is standard for GPU programming (single precision).
    Args:
        n: Size of the matrices (N x N).
        dtype: Data type of the matrices (default: np.float32).
    Returns: 
        Two N x N matrices A and B.
    """
    
    A = np.random.rand(n, n).astype(dtype)
    B = np.random.rand(n, n).astype(dtype)
    return A, B

def check_correctness(target: np.ndarray, reference: np.ndarray, tolerance: float = 1e-4) -> bool:
    """
    Compares two matrices using NumPy's allclose.
    Args:
        target: The matrix to test.
        reference: The reference matrix.
        tolerance: The tolerance for comparison (default: 1e-4).
    Returns:
        True if matrices are close within the given tolerance, False otherwise.
    """
    
    if hasattr(target, 'get'): 
        target = target.get()
    if hasattr(reference, 'get'): 
        reference = reference.get()
    try:
        np.testing.assert_allclose(target, reference, atol=tolerance, rtol=tolerance)
        return True
    except AssertionError:
        return False

def benchmark_function(func, name: str, *args, n_iter: int = 5) -> Tuple[np.ndarray, float]:
    """
    Benchmarks a given function by running it multiple times and measuring execution time.
    Args:
        func: The function to benchmark.
        name: Name of the benchmark (for reporting).
        *args: Arguments to pass to the function.
        n_iter: Number of iterations to run (default: 5).
    Returns:
        A tuple containing the result of the function and the average execution time in milliseconds.
    """
    func(*args) # warm-up
    cp.cuda.Device(0).synchronize()
    
    timings = []
    result = None

    for i in range(n_iter):
        cp.cuda.Device(0).synchronize()
        
        start_time = time.perf_counter()
        result = func(*args) 
        
        cp.cuda.Device(0).synchronize()
        end_time = time.perf_counter()
        
        timings.append((end_time - start_time) * 1000)

    avg_time = np.mean(timings)
    std_dev = np.std(timings)
    
    print(f"[{name}] Avg Time: {avg_time:.4f} ms (±{std_dev:.2f} ms) | Runs: {n_iter}")
    
    return result, avg_time

Overwriting src/utils.py


In [3]:
%%writefile src/cpu_baseline.py
import numpy as np

def cpu_matmul(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    """
    Standard Matrix Multiplication using Triple Nested Loops.
    C[i][j] = sum(A[i][k] * B[k][j])
    Args:
        A: First input matrix.
        B: Second input matrix.
    Returns:
        The resulting matrix after multiplication C = A * B.
    """

    A = np.array(A)
    B = np.array(B)
    
    rows_A, cols_A = A.shape
    rows_B, cols_B = B.shape
    
    if cols_A != rows_B:
        raise ValueError("Cannot multiply: Dimensions do not match.")
        
    C = np.zeros((rows_A, cols_B), dtype=A.dtype)
    
    for i in range(rows_A):          
        for j in range(cols_B):      
            total = 0
            for k in range(cols_A):  
                total += A[i, k] * B[k, j]
            C[i, j] = total
            
    return C

Overwriting src/cpu_baseline.py


In [4]:
%%writefile src/gpu_ops.py


import cupy as cp
import numpy as np
import os

def transfer_to_gpu(A_host: np.ndarray, B_host: np.ndarray) -> tuple:
    """
    Transfers numpy arrays from Host (CPU) to Device (GPU).
    Args:
        A_host: First input matrix on host (CPU).
        B_host: Second input matrix on host (CPU).
    Returns:
        Two matrices A and B on device (GPU).
    """
    A_gpu = cp.asarray(A_host)
    B_gpu = cp.asarray(B_host)
    return A_gpu, B_gpu

def cupy_matmul_library(A_gpu: cp.ndarray, B_gpu: cp.ndarray) -> cp.ndarray:
    """
    Performs Matrix Multiplication using CuPy's optimized library.
    Args:
        A_gpu: First input matrix on device (GPU).
        B_gpu: Second input matrix on device (GPU).
    Returns:    
        The resulting matrix after multiplication C = A * B on device (GPU).
    """
    return cp.matmul(A_gpu, B_gpu)

def run_custom_kernel(A_gpu: cp.ndarray, B_gpu: cp.ndarray, N: int, block_size=(16, 16)) -> cp.ndarray:
    """
    Runs a custom naive matrix multiplication kernel.
    Args:
        A_gpu: First input matrix on device (GPU).
        B_gpu: Second input matrix on device (GPU).
        N: Size of the NxN matrices.    
        block_size: The block size to use in the kernel launch (default: (16,16)).
    Returns:    
        The resulting matrix C = A * B on device (GPU).
    """

    with open('kernels/matmul.cu', 'r') as f:
        kernel_code = f.read()
    
    kernel = cp.RawKernel(kernel_code, 'matmul_kernel')
    
    C_gpu = cp.zeros((N, N), dtype=cp.float32)
    
    grid_x = (N + block_size[0] - 1) // block_size[0]
    grid_y = (N + block_size[1] - 1) // block_size[1]
    grid_dim = (grid_x, grid_y)
    
    kernel(grid_dim, block_size, (A_gpu, B_gpu, C_gpu, cp.int32(N)))
    
    return C_gpu
    
def run_tiled_kernel_dynamic(A_gpu: cp.ndarray, B_gpu: cp.ndarray, N: int, tile_size: int =16) -> cp.ndarray:
    """
    Runs a tiled matrix multiplication kernel with dynamic tile size.
    Args:
        A_gpu: First input matrix on device (GPU).
        B_gpu: Second input matrix on device (GPU).
        N: Size of the NxN matrices.
        tile_size: The tile size to use in the kernel.
    Returns:    
        The resulting matrix C = A * B on device (GPU).
    """

    with open('kernels/tiled_matmul.cu', 'r') as f:
        raw_code = f.read()
    

    augmented_code = f"#define TILE_WIDTH {tile_size}\n" + raw_code
    
    kernel = cp.RawKernel(augmented_code, 'tiled_matmul_kernel')
    
    C_gpu = cp.zeros((N, N), dtype=cp.float32)
    

    block_dim = (tile_size, tile_size)
    grid_x = (N + tile_size - 1) // tile_size
    grid_y = (N + tile_size - 1) // tile_size
    
    kernel((grid_x, grid_y), block_dim, (A_gpu, B_gpu, C_gpu, cp.int32(N)))
    
    return C_gpu

Overwriting src/gpu_ops.py


In [5]:
%%writefile kernels/matmul.cu
extern "C" {
    __global__ void matmul_kernel(const float* A, const float* B, float* C, int N) {
        

        int row = blockIdx.y * blockDim.y + threadIdx.y;
        int col = blockIdx.x * blockDim.x + threadIdx.x;

        if (row < N && col < N) {
            
            float sum = 0.0f;
            
            for (int k = 0; k < N; k++) {
                float a = A[row * N + k];
                float b = B[k * N + col];
                sum += a * b;
            }

            C[row * N + col] = sum;
        }
    }
}

Overwriting kernels/matmul.cu


In [6]:
%%writefile kernels/tiled_matmul.cu
extern "C" {

    __global__ void tiled_matmul_kernel(const float* A, const float* B, float* C, int N) {
        
        __shared__ float As[TILE_WIDTH][TILE_WIDTH];
        __shared__ float Bs[TILE_WIDTH][TILE_WIDTH];

        int bx = blockIdx.x;  int by = blockIdx.y;
        int tx = threadIdx.x; int ty = threadIdx.y;

        int row = by * TILE_WIDTH + ty;
        int col = bx * TILE_WIDTH + tx;

        float value = 0.0f;

        int num_phases = (N + TILE_WIDTH - 1) / TILE_WIDTH;

        for (int m = 0; m < num_phases; ++m) {


            
            int col_A = m * TILE_WIDTH + tx;
            if (row < N && col_A < N)
                As[ty][tx] = A[row * N + col_A];
            else
                As[ty][tx] = 0.0f;

            int row_B = m * TILE_WIDTH + ty;
            if (row_B < N && col < N)
                Bs[ty][tx] = B[row_B * N + col];
            else
                Bs[ty][tx] = 0.0f;

            __syncthreads();


            for (int k = 0; k < TILE_WIDTH; ++k) {
                value += As[ty][k] * Bs[k][tx];
            }

            __syncthreads();
        }

        if (row < N && col < N) {
            C[row * N + col] = value;
        }
    }
}

Overwriting kernels/tiled_matmul.cu


## File Imports

In [7]:
import sys
import os
import numpy as np
import cupy as cp

from src.utils import generate_matrices, check_correctness, benchmark_function
from src.cpu_baseline import cpu_matmul
from src.gpu_ops import transfer_to_gpu, cupy_matmul_library, run_custom_kernel, run_tiled_kernel_dynamic

print("Modules imported successfully.")
N = 256

Modules imported successfully.


## CPU Baseline

In [8]:

print(f"Part 1: CPU Baseline Benchmark (N={N})")

A_host, B_host = generate_matrices(N)


C_cpu, time_cpu = benchmark_function(cpu_matmul, "CPU Naive", A_host, B_host, n_iter=5)

C_ref = np.dot(A_host, B_host)

if check_correctness(C_cpu, C_ref):
    print("PASS: CPU implementation matches NumPy reference.")
else:
    print("FAIL: CPU implementation is incorrect.")

Part 1: CPU Baseline Benchmark (N=256)
[CPU Naive] Avg Time: 6592.3083 ms (±428.07 ms) | Runs: 5
PASS: CPU implementation matches NumPy reference.


## GPU using CuPy

In [9]:
print("Part 2: CuPy (GPU) Implementation")

print(f"\nExperiment A: Small Matrix (N={N})")


print("Transferring data to GPU...", end=" ")
A_gpu, B_gpu = transfer_to_gpu(A_host, B_host) 
print("Done.")

print("Warming up GPU...", end=" ")
cupy_matmul_library(A_gpu, B_gpu)
cp.cuda.Stream.null.synchronize()
print("Done.")

C_gpu, time_gpu = benchmark_function(cupy_matmul_library, "CuPy Library", A_gpu, B_gpu, n_iter=5)


if check_correctness(C_gpu, C_ref):
    print("PASS: CuPy result matches Reference.")
    print(f"Speedup vs CPU: {time_cpu / time_gpu:.2f}x")
else:
    print("FAIL: CuPy result incorrect.")


N_large = 2000
print(f"\nExperiment B: Large Matrix (N={N_large})")

A_large, B_large = generate_matrices(N_large)
A_large_gpu, B_large_gpu = transfer_to_gpu(A_large, B_large)

C_large_gpu, time_large_gpu = benchmark_function(cupy_matmul_library, f"CuPy (N={N_large})", A_large_gpu, B_large_gpu, n_iter=5)

Part 2: CuPy (GPU) Implementation

Experiment A: Small Matrix (N=256)
Transferring data to GPU... Done.
Warming up GPU... Done.
[CuPy Library] Avg Time: 0.1976 ms (±0.06 ms) | Runs: 5
PASS: CuPy result matches Reference.
Speedup vs CPU: 33355.74x

Experiment B: Large Matrix (N=2000)
[CuPy (N=2000)] Avg Time: 6.0550 ms (±0.06 ms) | Runs: 5


## Custom Kernel 

In [30]:
print("Part 3: Custom Kernel Benchmark")

N = 2000 
print(f"Matrix Size: {N}x{N}")

A_host, B_host = generate_matrices(N)
A_gpu, B_gpu = transfer_to_gpu(A_host, B_host)

block_sizes = [(8, 8), (16, 16), (32, 32)]

for bs in block_sizes:
    print(f"\n--- Testing Block Size: {bs} ---")
    
    func = lambda: run_custom_kernel(A_gpu, B_gpu, N, block_size=bs)
    C_custom, time_custom = benchmark_function(func, f"Naive Kernel {bs}", n_iter=5)
    
    C_ref_gpu = cupy_matmul_library(A_gpu, B_gpu)
    if check_correctness(C_custom, C_ref_gpu, tolerance=1e-3):
        print("Result Correct")
    else:
        print("Result Incorrect")

print("\n--- Comparison ---")

_, time_lib = benchmark_function(cupy_matmul_library, "CuPy Library", A_gpu, B_gpu, n_iter=5)
print(f"Library is {time_custom/time_lib:.2f}x faster than the Naive Kernel.")

Part 3: Custom Kernel Benchmark
Matrix Size: 2000x2000

--- Testing Block Size: (8, 8) ---
[Naive Kernel (8, 8)] Avg Time: 67.3351 ms (±3.37 ms) | Runs: 5
Result Correct

--- Testing Block Size: (16, 16) ---
[Naive Kernel (16, 16)] Avg Time: 40.8900 ms (±3.92 ms) | Runs: 5
Result Correct

--- Testing Block Size: (32, 32) ---
[Naive Kernel (32, 32)] Avg Time: 32.8921 ms (±3.97 ms) | Runs: 5
Result Correct

--- Comparison ---
[CuPy Library] Avg Time: 2.5780 ms (±0.02 ms) | Runs: 5
Library is 12.76x faster than the Naive Kernel.


## Memory Tiling

In [32]:
print("Part 4: Tiling Optimization (Dynamic)")
N = 2000
print(f"Matrix Size: {N}x{N}")

A_host, B_host = generate_matrices(N)
A_gpu, B_gpu = transfer_to_gpu(A_host, B_host)

print("\n--- Baseline: Naive ---")
_, time_naive = benchmark_function(
    lambda: run_custom_kernel(A_gpu, B_gpu, N, block_size=(16,16)), 
    "Naive (16x16)",
    n_iter=5
)

tile_configs = [16, 32]
results = {}

print("\n--- Tiling Experiments ---")
for size in tile_configs:
    name = f"Tiled ({size}x{size})"
    
    C_tiled, time_tiled = benchmark_function(
        lambda: run_tiled_kernel_dynamic(A_gpu, B_gpu, N, tile_size=size), 
        name,
        n_iter=5
    )
    
    C_ref = cupy_matmul_library(A_gpu, B_gpu)
    if check_correctness(C_tiled, C_ref, tolerance=1e-3):
        print(f"{name} Validated")
        results[size] = time_tiled
    else:
        print(f"{name} FAILED")

print("\n=== Final Analysis ===")
best_tile = min(results, key=results.get)
print(f"Best Configuration: Tiled {best_tile}x{best_tile}")
print(f"Speedup (Tiled vs Naive): {time_naive / results[best_tile]:.2f}x")



Part 4: Tiling Optimization (Dynamic)
Matrix Size: 2000x2000

--- Baseline: Naive ---
[Naive (16x16)] Avg Time: 40.0066 ms (±4.56 ms) | Runs: 5

--- Tiling Experiments ---
[Tiled (16x16)] Avg Time: 27.3159 ms (±0.11 ms) | Runs: 5
Tiled (16x16) Validated
[Tiled (32x32)] Avg Time: 20.8915 ms (±2.33 ms) | Runs: 5
Tiled (32x32) Validated

=== Final Analysis ===
Best Configuration: Tiled 32x32
Speedup (Tiled vs Naive): 1.91x


## Final Comparison

In [37]:
print("Final High-Standard Benchmark (Avg of 5 Runs)")
N = 2000
A_host, B_host = generate_matrices(N)
A_gpu, B_gpu = transfer_to_gpu(A_host, B_host)

C_naive, time_naive = benchmark_function(
    lambda: run_custom_kernel(A_gpu, B_gpu, N, block_size=(16,16)), 
    "Naive (16x16)",
    n_iter=5
)

C_tiled_16, time_tiled_16 = benchmark_function(
    lambda: run_tiled_kernel_dynamic(A_gpu, B_gpu, N, tile_size=16), 
    "Tiled (16x16)",
    n_iter=5
)

C_tiled_32, time_tiled_32 = benchmark_function(
    lambda: run_tiled_kernel_dynamic(A_gpu, B_gpu, N, tile_size=32), 
    "Tiled (32x32)",
    n_iter=5
)

C_lib, time_lib = benchmark_function(
    lambda: cupy_matmul_library(A_gpu, B_gpu), 
    "CuPy Library",
    n_iter=10  
)


print("\nFinal Results Summary:")
print(f"{'Method':<20} | {'Time (ms)':<15} | {'Speedup vs Naive':<20}")
print("-" * 60)
print(f"{'Naive (16x16)':<20} | {time_naive:<15.4f} | {'1.00x':<20}")
print(f"{'Tiled (16x16)':<20} | {time_tiled_16:<15.4f} | {time_naive/time_tiled_16:<20.2f}")
print(f"{'Tiled (32x32)':<20} | {time_tiled_32:<15.4f} | {time_naive/time_tiled_32:<20.2f}")
print(f"{'CuPy Library':<20} | {time_lib:<15.4f} | {time_naive/time_lib:<20.2f}")

Final High-Standard Benchmark (Avg of 5 Runs)
[Naive (16x16)] Avg Time: 39.6574 ms (±5.11 ms) | Runs: 5
[Tiled (16x16)] Avg Time: 26.4581 ms (±0.09 ms) | Runs: 5
[Tiled (32x32)] Avg Time: 21.7216 ms (±0.93 ms) | Runs: 5
[CuPy Library] Avg Time: 3.4752 ms (±0.36 ms) | Runs: 10

Final Results Summary:
Method               | Time (ms)       | Speedup vs Naive    
------------------------------------------------------------
Naive (16x16)        | 39.6574         | 1.00x               
Tiled (16x16)        | 26.4581         | 1.50                
Tiled (32x32)        | 21.7216         | 1.83                
CuPy Library         | 3.4752          | 11.41               
